# Pydata Global 2022: Production-grade Machine Learning with Flyte

In this tutorial, you're going to learn about some of the key challenges to building and deploying reliable machine learning systems. At a high level, these challenges are the following:

- Scalability
- Data Quality
- Reproducibility
- Recoverability
- Auditability

In [10]:
import flytekit
from flytekit.remote import FlyteRemote
from flytekit.configuration import Config


remote = FlyteRemote(
    config=Config.auto(config_file="./config.yaml"),
    default_project="flytesnacks",
    default_domain="development",
)

## Introduction

### Environment Setup

Follow the instructions in the [setup instructions](./README.md#setup) of
the README.

### Example 0: Flyte Basics

Let's take a look at the [first example](./workflows/example_00_intro.py).

In it, you'll see a simple pipeline that uses the penguins dataset to train a
penguin species classifier. You can run this workflow locally with:

```
python workflows/example_00_intro.py
```

#### Exercise: Understanding Workflows

Workflows are basically a domain-specific language (DSL) that builds an
execution graph that uses tasks as the building blocks for more complex pipelines.

Insert a debugging breakpoint `import pdb; pdb.set_trace()` on line 80 of the
`example_00_intro.py` script and rerun it. Take a look at all the variables
in the `training_workflow` like `data` and `model`. What data type are they?

#### Registering Your Workflow

Once you're happy with the state of your tasks and workflows, you can register
them by first packaging them up into a portable flyte archive:

```
export IMAGE='ghcr.io/flyteorg/flyte-conference-talks:pydata-global-2022-latest'
pyflyte --pkgs workflows package --image $IMAGE -f
```

This will create a `flyte-package.tgz` archive file that contains the serialized
tasks and workflows in this project. Then, you can register it with:

```
flytectl register files --project flytesnacks --domain development --archive flyte-package.tgz --version v0
```

Now we can go over to https://sandbox.union.ai/console
(or http://localhost:30080/console if you're using a local Flyte cluster) to
check out the tasks and workflows we just registered.

In [4]:
from workflows import example_00_intro

execution = remote.execute_local_workflow(
    example_00_intro.training_workflow,
    inputs={
        "hyperparameters": {"C": 0.1, "max_iter": 5000},
        "test_size": 0.2,
        "random_state": 11,
    }
)
remote.generate_console_url(execution)

'http://localhost:30080/console/projects/flytesnacks/domains/development/executions/fdf00ff99f65b45cda38'

In [5]:
execution = remote.wait(execution)

In [7]:
clf = execution.outputs["o0"]
clf

LogisticRegression(C=0.1, max_iter=5000.0)

#### Execise: Understanding Workflows

Workflows are basically a domain-specific language (DSL) that builds an
execution graph that uses tasks as the building blocks for more complex pipelines.

Insert a debugging breakpoint `import pdb; pdb.set_trace()` on line 80 of the
`example_00_intro.py` script and rerun it.

#### Scheduling Launchplans

Activate the schedule:

In [19]:
lp_id = remote.fetch_launch_plan(name="scheduled_training_workflow").id
remote.client.update_launch_plan(lp_id, "ACTIVE")
print("activated scheduled_training_workflow")

Get the execution for the most recent schedule run.

In [ ]:
recent_executions = [
    execution
    for execution in remote.recent_executions()
    if execution.spec.launch_plan.name == "scheduled_training_workflow"
]

scheduled_execution = None
model = None
if recent_executions:
    scheduled_execution = recent_executions[0]
    print(scheduled_execution)
    scheduled_execution = remote.wait(scheduled_execution)
    model = scheduled_execution.outputs["o0"]
    model

print(model)

Now deactivate the schedule

In [44]:
remote.client.update_launch_plan(lp_id, "INACTIVE")
print("deactivated scheduled_training_workflow")

### Example 1: Dynamic Workflows

Dynamic workflows allow you to create execution graphs on the fly. This allows
you to specify for loops over inputs to implement a grid search model tuning
workflow.

In [ ]:
from workflows import example_01_dynamic

execution = remote.execute_local_workflow(
    example_01_dynamic.tuning_workflow,
    inputs={
        "hyperparam_grid": [
            {"C": 0.1, "max_iter": 5000},
            {"C": 0.01, "max_iter": 5000},
            {"C": 0.001, "max_iter": 5000},
        ],
    }
)
remote.generate_console_url(execution)

### Example 2: Map Tasks

In [ ]:
from workflows import example_02_map_task

execution = remote.execute_local_workflow(
    example_02_map_task.tuning_workflow,
    inputs={
        "hyperparam_grid": [
            {"C": 0.1},
            {"C": 0.01},
            {"C": 0.001},
        ],
    }
)
print(remote.generate_console_url(execution))
execution = remote.wait(execution)

### Example 3: Plugins

### Example 4: Type System

### Example 5: Pandera Types

### Example 6: Reproducibility

### Example 7: Caching

### Example 8: Recovering Failed Executions

### Example 9: Checkpointing

### Example 10: Visiualization with Flyte Decks

In [86]:
from workflows import example_10_flyte_decks


def download_deck(execution, local_path: str):
    remote.file_access.download(
        execution.node_executions["n1"].closure.deck_uri, local_path
    )
    print(f"Flyte decks for execution {execution.id.name} downloaded to {local_path}")


execution = remote.execute_local_workflow(
    example_10_flyte_decks.penguins_data_workflow,
    inputs={},
)
print(remote.generate_console_url(execution))
execution = remote.wait(execution)

In [87]:
download_deck(execution, "decks/example_10_decks.html")

Flyte decks for execution f381cfa5d070545a4ae6 downloaded to decks/example_10_decks.html


### Example 11: Extending Flyte Decks